In [1]:
import pandas as pd
import datetime as dt
import geopandas as gp
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import re
from shapely.geometry import Point
%matplotlib inline

In [3]:
MTA_TURNSTILE_URL = "http://web.mta.info/developers/turnstile.html"
MTA_FILE_ROOT_URL = "http://web.mta.info/developers/"

In [4]:
# '''
# slightly modified from https://github.com/piratefsh/mta-turnstile-scraper/blob/master/mta_scraper.py'''

# def get_site():
#     f = urllib2.urlopen(MTA_TURNSTILE_URL)
#     content = f.read()
#     return content

# def get_turnstile_links(html):
#     soup = BeautifulSoup(html, 'html.parser')
#     links = soup.find_all('a')
#     turnstile_links = [(link.text, MTA_FILE_ROOT_URL + link['href']) for link in links if re.match('.*day.*20..', link.text)]
#     return turnstile_links 

# def get_links_by_date(start, end):
#     links = get_links()
#     in_range = []
#     for text,link in links:
#         date = datetime.strptime(text , "%A, %B %d, %Y") 
#         if date >= start and date <= end:
#             in_range.append((text, link))
    
#     return in_range

# def get_links():
#     return get_turnstile_links(get_site())

In [5]:
metadata = 'http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description_pre-10-18-2014.txt'
f = urllib2.urlopen(metadata)
content = f.readlines()
col_names_14 = str(content[2]).split(',')

def turnstile_df(links, post2014 = True): 
    '''takes links and returns dataframe with all data in one
    pre 2014 was captured differently and thus requires a second argument for correct columns'''
    df = pd.DataFrame()
    if post2014 == True: 
        for text, link in links: 
            temp_df = pd.read_csv(link)
            temp_df['DATE'] = temp_df['DATE'] + ' ' + temp_df['TIME']
            temp_df['DATE'] = pd.to_datetime(temp_df['DATE'])
            df = df.append(temp_df, ignore_index = True)
    elif post2014 == False: 
        for text, link in links: 
            temp_df = pd.read_csv(link, header = None, names = col_names_14, parse_dates = True,
                                 infer_datetime_format = True)
            df = df.append(temp_df, ignore_index = True)
    return df

In [6]:
# post 18 oct 2014 to 2016
a = get_site()
b = get_turnstile_links(a)
start = datetime(2014, 10, 18)
end = datetime(2016, 12, 31)
desired = get_links_by_date(start, end)
y1416 = turnstile_df(desired, post2014= True)

In [11]:
y1416['DATE'] = y1416['DATE'] + ' ' + y1416['TIME']
y1416['DATE'] = pd.to_datetime(y1416['DATE'])

In [25]:
y1416.to_csv("../Data/subway/subways_1416.csv")

In [2]:
data = pd.read_csv("../Data/subways_1416.csv", parse_dates = ['DATE'])

In [3]:
data = data[(data.DIVISION != 'PTH') & (data.DIVISION != 'SRT') & (data.DIVISION != 'RIT')]

In [4]:
data.drop(['Unnamed: 0'],
        inplace = True, axis = 1)

In [5]:
data.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace=True)

In [6]:
data.loc[:,'uni'] = data.loc[:,'STATION'] + " " + data.loc[:,'LINENAME'] + " " + data.loc[:,'DIVISION']

In [7]:
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES', u'EXITS', u'uni'],
      dtype='object')

In [8]:
df = data[data.DESC == 'REGULAR']
df = df.groupby([u'C/A', u'UNIT', u'uni', u'SCP', u'DATE']).sum()

In [9]:
df["ENTRIES_DIFF"] = df.groupby(level=[0,1,2,3])['ENTRIES'].transform(lambda x: abs(x - x.shift()))
df["EXITS_DIFF"] = df.groupby(level=[0,1,2,3])['EXITS'].transform(lambda x: abs(x - x.shift()))

In [10]:
df.dropna(inplace=True)

In [11]:
df.reset_index(inplace=True)

In [12]:
df.ENTRIES_DIFF.quantile(0.9999)

20100.609599977732

In [13]:
reduced = df[df.ENTRIES_DIFF < df.ENTRIES_DIFF.quantile(0.9999)]['ENTRIES_DIFF']
    
lower_thresh = 0
upper_thresh = reduced.mean() + 3 * reduced.std()
                                                                           
outlier_mask = df['ENTRIES_DIFF'].map(lambda x: lower_thresh <= x <= upper_thresh)
    
df = df[outlier_mask]

In [14]:
df = df.set_index('DATE').groupby([pd.TimeGrouper('D'), 'uni']).sum()

In [15]:
df.reset_index(['uni'], inplace=True)

In [16]:
df.drop(['ENTRIES', 'EXITS'], axis=1, inplace=True)

In [17]:
df.reset_index(inplace=True)

In [18]:
df.columns

Index([u'DATE', u'uni', u'ENTRIES_DIFF', u'EXITS_DIFF'], dtype='object')

In [19]:
stations = df['uni']

In [20]:
stations.to_csv("../Data/subway/stations.csv")

In [21]:
# df.DATE.describe()

# Subways

In [22]:
subways = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv',
                     usecols=['Station_Name', 'Station_Latitude','Station_Longitude', 'Division','Route_1', 'Route_2', 
                              'Route_3', 'Route_4', 'Route_5', 'Route_6', 'Route_7', 'Route_8', 'Route_9', 'Route_10',
                              'Route_11'])

In [23]:
for i in range(len(subways)):
    
    if ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
        (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
        (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
        (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
        (str(subways.loc[i,'Route_10']) != 'nan') & (str(subways.loc[i,'Route_11']) != 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) + str(subways.loc[i,'Route_10']) + 
                                        str(subways.loc[i,'Route_11']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
          (str(subways.loc[i,'Route_10']) != 'nan') &(str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) + str(subways.loc[i,'Route_10']))

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') & 
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) != 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) + 
                                        str(subways.loc[i,'Route_9']) )
    
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) != 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']) + str(subways.loc[i,'Route_8']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) != 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) + 
                                        str(subways.loc[i,'Route_7']))    

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) != 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) + str(subways.loc[i,'Route_6']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) != 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + str(subways.loc[i,'Route_4']) + 
                                        str(subways.loc[i,'Route_5']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) != 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) + 
                                        str(subways.loc[i,'Route_4']) )

    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) != 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) + 
                                        str(subways.loc[i,'Route_3']) )
        
    elif ((str(subways.loc[i,'Route_2']) != 'nan') & (str(subways.loc[i,'Route_3']) == 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']) + str(subways.loc[i,'Route_2']) )

    elif ((str(subways.loc[i,'Route_2']) == 'nan') & (str(subways.loc[i,'Route_3']) == 'nan') &
          (str(subways.loc[i,'Route_4']) == 'nan') & (str(subways.loc[i,'Route_5']) == 'nan') &
          (str(subways.loc[i,'Route_6']) == 'nan') & (str(subways.loc[i,'Route_7']) == 'nan') &
          (str(subways.loc[i,'Route_8']) == 'nan') & (str(subways.loc[i,'Route_9']) == 'nan') &
          (str(subways.loc[i,'Route_10']) == 'nan') & (str(subways.loc[i,'Route_11']) == 'nan')):
        subways.loc[i,'LINENAME'] = str(str(subways.loc[i,'Route_1']))

In [24]:
subways.loc[:,'uni'] = subways.loc[:,'Station_Name'] + " " + subways.loc[:,'LINENAME'] + " " + subways.loc[:,'Division']

In [25]:
subways.drop(['Route_1','Route_2','Route_3','Route_4','Route_5','Route_6','Route_7', 'Route_8', 'Route_9', 'Route_10',
              'Route_11'],inplace=True,axis=1)
subways.reset_index(drop=True, inplace=True)

In [26]:
subways.drop_duplicates(inplace = True)
subways.reset_index(drop=True, inplace=True)

In [27]:
subways.head()

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT
3,BMT,Broadway,40.761820,-73.925508,NQ,Broadway NQ BMT
4,BMT,36 Av-Washington Av,40.756804,-73.929575,NQ,36 Av-Washington Av NQ BMT


In [28]:
subways.to_csv("../Data/subway/subway_locations.csv")

In [29]:
len(subways)

474

# Subway Mapper

In [31]:
mapper = pd.read_csv("../Data/subway/stations_mapper.csv")

In [35]:
df.head(3)

,DATE,uni,ENTRIES_DIFF,EXITS_DIFF
0,2014-10-11,1 AVE L BMT,13150.0,15662.0
1,2014-10-11,103 ST 1 IRT,9486.0,3681.0
2,2014-10-11,103 ST 6 IRT,8584.0,7402.0


In [36]:
subways.head(3)

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT


In [37]:
mapper.head(3)

,stations,Map
0,103 ST 1 IRT,103rd St 1 IRT
1,103 ST 6 IRT,103rd St 6 IRT
2,103 ST-CORONA 7 IRT,103rd St 7 IRT


In [39]:
len(mapper.stations.unique())

716

In [41]:
df_merged = pd.merge(df, mapper, left_on='uni', right_on='stations', how='left')

In [50]:
df_merged.columns

Index([u'DATE', u'uni', u'ENTRIES_DIFF', u'EXITS_DIFF', u'stations', u'Map'], dtype='object')

In [54]:
df_merged.dropna(inplace=True)

In [56]:
df_merged.head()

,DATE,uni,ENTRIES_DIFF,EXITS_DIFF,stations,Map
0,2014-10-11,1 AVE L BMT,13150.0,15662.0,1 AVE L BMT,1st Av L BMT
1,2014-10-11,103 ST 1 IRT,9486.0,3681.0,103 ST 1 IRT,103rd St 1 IRT
2,2014-10-11,103 ST 6 IRT,8584.0,7402.0,103 ST 6 IRT,103rd St 6 IRT
3,2014-10-11,103 ST BC IND,2266.0,2193.0,103 ST BC IND,103rd St BC IND
4,2014-10-11,103 ST-CORONA 7 IRT,13558.0,9128.0,103 ST-CORONA 7 IRT,103rd St 7 IRT


In [58]:
len(df_merged)

364104

In [57]:
subways.head(3)

,Division,Station_Name,Station_Latitude,Station_Longitude,LINENAME,uni
0,BMT,Ditmars Blvd,40.775036,-73.912034,NQ,Ditmars Blvd NQ BMT
1,BMT,Astoria Blvd-Hoyt Av,40.770258,-73.917843,NQ,Astoria Blvd-Hoyt Av NQ BMT
2,BMT,30 Av-Grand Av,40.766779,-73.921479,NQ,30 Av-Grand Av NQ BMT


In [59]:
df_merged = pd.merge(df_merged, subways, left_on='Map', right_on='uni')

In [60]:
len(df_merged)

377325

In [63]:
df_merged.columns

Index([u'DATE', u'uni_x', u'ENTRIES_DIFF', u'EXITS_DIFF', u'stations', u'Map',
       u'Division', u'Station_Name', u'Station_Latitude', u'Station_Longitude',
       u'LINENAME', u'uni_y'],
      dtype='object')

In [64]:
df_merged.drop([u'uni_x',u'stations', u'Map',
                u'Division', u'LINENAME', u'uni_y'], inplace = True, axis = 1)

# Taxi Zone

In [69]:
tzones = gp.read_file("../Data/taxi_zones/taxi_zones_updated.shp")

In [75]:
tzones.to_crs(epsg=4326, inplace=True)

In [71]:
df_merged.head(2)

,DATE,ENTRIES_DIFF,EXITS_DIFF,Station_Name,geometry
0,2014-10-11,13150.0,15662.0,1st Av,POINT (-73.981628 40.730953)
1,2014-10-12,17117.0,20513.0,1st Av,POINT (-73.981628 40.730953)


In [76]:
tzones.head(3)

,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,geometry,zone
0,1,1,0.000782,0.116357,EWR,POLYGON ((-74.18445300000005 40.69499600003324...,Newark Airport
1,2,2,0.004866,0.433470,Queens,(POLYGON ((-73.82337597260654 40.6389870472100...,Jamaica Bay
2,3,3,0.000314,0.084341,Bronx,POLYGON ((-73.84792614099977 40.87134223403326...,Allerton/Pelham Gardens


In [81]:
df_merged['DATE'] = pd.to_datetime(df_merged["DATE"])

crs = {'init': 'epsg:4326', 'no_defs': True}
geometry = [Point(xy) for xy in zip(df_merged["Station_Longitude"], df_merged["Station_Latitude"])]
df_merged = df_merged.drop(["Station_Longitude", "Station_Latitude"], axis=1)
df_merged = gp.GeoDataFrame(df_merged, crs=crs, geometry=geometry)

print "Spatially joining data and taxi zones"

data_geo = gp.sjoin(tzones, df_merged) 
data_geo.drop(['borough', u'OBJECTID', 'Shape_Area', 'Shape_Leng',
                   u'geometry', 'index_right', 'zone', 'Station_Name'], axis = 1, inplace = True)   
    

data_agg = data_geo.set_index('DATE').groupby([pd.TimeGrouper('D'),'LocationID']).agg({'ENTRIES_DIFF':'sum', 
                                                                                      'EXITS_DIFF': 'sum'})
data_agg.reset_index('LocationID', inplace=True)

In [82]:
data_agg.head(3)

,LocationID,EXITS_DIFF,ENTRIES_DIFF
DATE,,,
2014-10-11,3,280.0,10.0
2014-10-11,7,27564.0,31367.0
2014-10-11,14,8609.0,16793.0


In [83]:
len(data_agg.LocationID.unique())

152

In [84]:
data_agg.to_csv("../Data/Aggregated/subways.csv")

In [85]:
data_agg.columns

Index([u'LocationID', u'EXITS_DIFF', u'ENTRIES_DIFF'], dtype='object')

In [87]:
data_agg.reset_index(inplace=True)

In [88]:
data_agg.DATE.describe()

count                  121999
unique                    812
top       2016-04-04 00:00:00
freq                      152
first     2014-10-11 00:00:00
last      2016-12-30 00:00:00
Name: DATE, dtype: object